# Importing the Libraries

In [6]:
%matplotlib inline
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import cv2
import pickle

# Preprocessing 
- Loading images into arrays
- storing the arrays into pickle file

## DO NOT RUN THE CELLS UPTILL CNN MODEL

In [9]:
folder = "D:\\"
image_height = 480
image_width = 680

In [16]:
train_data_file = pd.read_csv(folder+'training.csv')

In [5]:
train_data_folder = os.path.join(folder,"images")

In [6]:
image_arr = []
for i in train_data_file['image_name'].values:
    try:
        image_path = os.path.join(train_data_folder,i)
        image = cv2.imread(image_path,cv2.IMREAD_UNCHANGED)
        resized_image = cv2.resize(image,(128,128))
        image_arr.append(resized_image)
    except:
        pass

In [25]:
with open("training_images_arr.pkl",'wb') as file:
    pickle.dump(image_arr,file)

In [8]:
j=0
for i in image_arr[:10]:
    j+=1
    plt.subplot(2,5,j)
    cv2.imshow("image",i)
    plt.show()
    

# CNN Model
- run the cells below this

In [2]:
from keras.layers import Input,Dense,Dropout,Conv2D,Flatten,MaxPooling2D,GlobalMaxPooling2D
from keras.models import Model
from keras.activations import relu,sigmoid,softmax
from keras.layers import BatchNormalization
from keras import optimizers
from keras.callbacks import ModelCheckpoint,EarlyStopping,TensorBoard
from keras.regularizers import l2,l1
from keras.utils import plot_model
import tensorflow as tf

Using TensorFlow backend.


In [3]:
inputs = Input(shape=(128,128,3))
localizer = Conv2D(64,(3,3),kernel_initializer="glorot_normal",bias_initializer="zeros",activation="relu",\
                   kernel_regularizer=l2(0.01),activity_regularizer=l2(0.01))(inputs)
localizer = BatchNormalization()(localizer)
localizer = Conv2D(64,(3,3),activation="relu",\
                   kernel_regularizer=l2(0.01),activity_regularizer=l2(0.01))(localizer)
localizer = BatchNormalization()(localizer)
localizer = MaxPooling2D(pool_size=3,strides=2,padding="same")(localizer)

localizer = Conv2D(128,(1,1),activation="relu",kernel_regularizer=l2(0.01),activity_regularizer=l2(0.01))(localizer)
localizer = BatchNormalization()(localizer)
localizer = Conv2D(128,(1,1),activation="relu",kernel_regularizer=l2(0.01),activity_regularizer=l2(0.01))(localizer)
localizer = BatchNormalization()(localizer)
localizer = MaxPooling2D(3,2,padding="same")(localizer)

localizer = Flatten()(localizer)
localizer = Dense(1024,activation="relu",kernel_regularizer=l2(0.01),activity_regularizer=l2(0.01))(localizer)
localizer = Dense(1024,activation="relu",kernel_regularizer=l2(0.01),activity_regularizer=l2(0.01))(localizer)
output = Dense(4)(localizer)

model = Model(inputs=inputs,outputs=output)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 126, 126, 64)      1792      
_________________________________________________________________
batch_normalization_1 (Batch (None, 126, 126, 64)      256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 124, 124, 64)      36928     
_________________________________________________________________
batch_normalization_2 (Batch (None, 124, 124, 64)      256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 62, 62, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 62, 62, 128)       8320      
__________

In [4]:
def metric_iou(y_true,y_pred):
    return tf.metrics.mean_iou(y_true,y_pred,4)

In [5]:
model.compile(optimizer='Adam',loss='mean_absolute_error')
folder="\\output\\weights.best.hdf5"
checkpoint = ModelCheckpoint(folder,monitor='loss',verbose=1,save_best_only=True,mode='min')
early_stopping = EarlyStopping(monitor='loss')
callbacks=[checkpoint,early_stopping]

# Preparing the Input Data

In [12]:
with open("training_images_arr.pkl","rb") as file:
    images_array = pickle.load(file)
images_array = np.array(images_array)

In [13]:
images_array = images_array[:4000]

In [14]:
normalized_arrays = images_array/255

In [15]:
normalized_arrays = normalized_arrays.astype("float32")

In [22]:
output_label = train_data_file[:4000].iloc[:,1:]
output_label = output_label.values

# Train the model

In [ ]:
model.fit(normalized_arrays,output_label,epochs=10,batch_size=32,callbacks=callbacks,verbose=0)

Epoch 1/10
